In [ ]:
import torch
import transformers
#使用bert进行分词
tokenizer = transformers.BertTokenizer.from_pretrained("hfl/chinese-macbert-large")
tokenizer.tokenize("上次住院")

In [ ]:
import jionlp

In [ ]:
jionlp.replace_entity("我吃莲藕排骨还喝汤，血糖升高，是不是不能喝啊")

In [ ]:
import jionlp as jio

In [ ]:
dataset_y = [[{'type': 'Person', 'text': '马成宇', 'offset': (0, 3)},
                  {'type': 'Company', 'text': '百度', 'offset': (10, 12)},
                  {'type': 'Company', 'text': '百度', 'offset': (20, 22)}],
                 [{'type': 'Company', 'text': '国力教育公司', 'offset': (2, 8)}],
                 [{'type': 'Organization', 'text': '延平区人民法院', 'offset': (0, 7)},
                  {'type': 'Company', 'text': '百度', 'offset': (10, 12)},
                  {'type': 'Company', 'text': '百度', 'offset': (20, 22)}]]
entity_dict = jio.ner.collect_dataset_entities(dataset_y)
print(entity_dict)

In [ ]:
replace_entity = jio.ReplaceEntity(entity_dict)
text = '腾讯致力于游戏，阿里巴巴致力于电商。小马会玩。'
entities = [{'type': 'Company', 'text': '腾讯', 'offset': (0, 2)},
                {'type': 'Company', 'text': '阿里巴巴', 'offset': (8, 12)},
                {'type': 'Person', 'text': '小马', 'offset': (18, 20)}]
aug_texts, aug_entities = replace_entity(text, entities)
print(aug_texts, aug_entities)

In [19]:
!nohup clash

nohup: ignoring input and appending output to 'nohup.out'


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_44209/2809784334.py:1 in <module>                                                 │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_44209/2809784334.py'                        │
│                                                                                                  │
│ /home/zhou/.conda/envs/medicalgpt/lib/python3.9/site-packages/ipykernel/zmqshell.py:633 in       │
│ system_piped                                                                                     │
│                                                                                                  │
│   630 │   │   │   │   │   cmd = "pushd %s &&%s" % (path, cmd)                                    │
│   631 │   │   │   │   self.user_ns["_exit_code"] = system(cmd)                                   │
│   632 │   │   else:                                                                              │
│ ❱ 633 │   │   │   self.user_ns["_exit_code"] = system(self.var_expand(cmd, depth=1))             │
│   634 │                                                                                          │
│   635 │   # Ensure new system_piped implementation is used                                       │
│   636 │   system = system_piped                                                                  │
│                                                                                                  │
│ /home/zhou/.conda/envs/medicalgpt/lib/python3.9/site-packages/IPython/utils/_process_posix.py:17 │
│ 9 in system                                                                                      │
│                                                                                                  │
│   176 │   │   │   │   # Ensure the subprocess really is terminated                               │
│   177 │   │   │   │   child.terminate(force=True)                                                │
│   178 │   │   # add isalive check, to ensure exitstatus is set:                                  │
│ ❱ 179 │   │   child.isalive()                                                                    │
│   180 │   │                                                                                      │
│   181 │   │   # We follow the subprocess pattern, returning either the exit status               │
│   182 │   │   # as a positive number, or the terminating signal as a negative                    │
│                                                                                                  │
│ /home/zhou/.conda/envs/medicalgpt/lib/python3.9/site-packages/pexpect/pty_spawn.py:705 in        │
│ isalive                                                                                          │
│                                                                                                  │
│   702 │   │                                                                                      │
│   703 │   │   ptyproc = self.ptyproc                                                             │
│   704 │   │   with _wrap_ptyprocess_err():                                                       │
│ ❱ 705 │   │   │   alive = ptyproc.isalive()                                                      │
│   706 │   │                                                                                      │
│   707 │   │   if not alive:                                                                      │
│   708 │   │   │   self.status = ptyproc.status                                                   │
│                                                                                                  │
│ /home/zhou/.conda/envs/medicalgpt/lib/python3.9/site-packages/ptyprocess/ptyprocess.py:711 in    │
│ isalive                                                    

In [24]:
!$env:http_proxy="http://127.0.0.1:7890"

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│  /tmp/ipykernel_44209/2610964502.py:1                                                            │
│ $env:http_proxy="http://127.0.0.1:7890"                                                          │
│ ▲                                                                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
SyntaxError: invalid syntax

In [21]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("THUDM/glm-10b", trust_remote_code=True)
model = AutoModelForSeq2SeqLM.from_pretrained("THUDM/glm-10b", trust_remote_code=True)
model = model.half().cuda()
model.eval()

# Inference
inputs = tokenizer("Ng is an adjunct professor at [MASK] (formerly associate professor and Director of its Stanford AI Lab or SAIL ). Also a pioneer in online education, Ng co-founded Coursera and deeplearning.ai.", return_tensors="pt")
inputs = tokenizer.build_inputs_for_generation(inputs, max_gen_length=512)
inputs = inputs.to('cuda')
outputs = model.generate(**inputs, max_length=512, eos_token_id=tokenizer.eop_token_id)
print(tokenizer.decode(outputs[0].tolist()))

# Training
inputs = tokenizer(
    ["Tsinghua University is located in [MASK].", "One minus one equals zero, is it correct? Answer: [MASK]"],
    return_tensors="pt", padding=True)
inputs = tokenizer.build_inputs_for_generation(inputs, targets=["Beijing", "No"], max_gen_length=8, padding=False)
inputs = inputs.to('cuda')
outputs = model(**inputs)
loss = outputs.loss
logits = outputs.logits

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/zhou/.conda/envs/medicalgpt/lib/python3.9/site-packages/urllib3/connectionpool.py:703 in   │
│ urlopen                                                                                          │
│                                                                                                  │
│    700 │   │   │   │   self._prepare_proxy(conn)                                                 │
│    701 │   │   │                                                                                 │
│    702 │   │   │   # Make the request on the httplib connection object.                          │
│ ❱  703 │   │   │   httplib_response = self._make_request(                                        │
│    704 │   │   │   │   conn,                                                                     │
│    705 │   │   │   │   method,                                                                   │
│    706 │   │   │   │   url,                                                                      │
│                                                                                                  │
│ /home/zhou/.conda/envs/medicalgpt/lib/python3.9/site-packages/urllib3/connectionpool.py:386 in   │
│ _make_request                                                                                    │
│                                                                                                  │
│    383 │   │                                                                                     │
│    384 │   │   # Trigger any extra validation we need to do.                                     │
│    385 │   │   try:                                                                              │
│ ❱  386 │   │   │   self._validate_conn(conn)                                                     │
│    387 │   │   except (SocketTimeout, BaseSSLError) as e:                                        │
│    388 │   │   │   # Py2 raises this as a BaseSSLError, Py3 raises it as socket timeout.         │
│    389 │   │   │   self._raise_timeout(err=e, url=url, timeout_value=conn.timeout)               │
│                                                                                                  │
│ /home/zhou/.conda/envs/medicalgpt/lib/python3.9/site-packages/urllib3/connectionpool.py:1042 in  │
│ _validate_conn                                                                                   │
│                                                                                                  │
│   1039 │   │                                                                                     │
│   1040 │   │   # Force connect early to allow us to validate the connection.                     │
│   1041 │   │   if not getattr(conn, "sock", None):  # AppEngine might not have  `.sock`          │
│ ❱ 1042 │   │   │   conn.connect()                                                                │
│   1043 │   │                                                                                     │
│   1044 │   │   if not conn.is_verified:                                                          │
│   1045 │   │   │   warnings.warn(                                                                │
│                                                                                                  │
│ /home/zhou/.conda/envs/medicalgpt/lib/python3.9/site-packages/urllib3/connection.py:419 in       │
│ connect                                                                                          │
│                                                                                                  │
│   416 │   │   ):                                                                                 │
│   417 │   │   │   context.load_default_certs()                                                   │
│   418 │   │                                                

In [17]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("BAAI/glm-10b-chinese", trust_remote_code=True)
model = AutoModelForSeq2SeqLM.from_pretrained("BAAI/glm-10b-chinese", trust_remote_code=True)
model = model.half().cuda()

inputs = tokenizer("凯旋门位于意大利米兰市古城堡旁。1807年为纪念[MASK]而建，门高25米，顶上矗立两武士青铜古兵车铸像。", return_tensors="pt")
inputs = tokenizer.build_inputs_for_generation(inputs, max_gen_length=512)
inputs = {key: value.cuda() for key, value in inputs.items()}
outputs = model.generate(**inputs, max_length=512, eos_token_id=tokenizer.eop_token_id)
print(tokenizer.decode(outputs[0].tolist()))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/zhou/.conda/envs/medicalgpt/lib/python3.9/site-packages/huggingface_hub/utils/_errors.py:2 │
│ 59 in hf_raise_for_status                                                                        │
│                                                                                                  │
│   256 │   </Tip>                                                                                 │
│   257 │   """                                                                                    │
│   258 │   try:                                                                                   │
│ ❱ 259 │   │   response.raise_for_status()                                                        │
│   260 │   except HTTPError as e:                                                                 │
│   261 │   │   error_code = response.headers.get("X-Error-Code")                                  │
│   262                                                                                            │
│                                                                                                  │
│ /home/zhou/.conda/envs/medicalgpt/lib/python3.9/site-packages/requests/models.py:1021 in         │
│ raise_for_status                                                                                 │
│                                                                                                  │
│   1018 │   │   │   )                                                                             │
│   1019 │   │                                                                                     │
│   1020 │   │   if http_error_msg:                                                                │
│ ❱ 1021 │   │   │   raise HTTPError(http_error_msg, response=self)                                │
│   1022 │                                                                                         │
│   1023 │   def close(self):                                                                      │
│   1024 │   │   """Releases the connection back to the pool. Once this method has been            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
HTTPError: 401 Client Error: Unauthorized for url: 
https://huggingface.co/BAAI/glm-10b-chinese/resolve/main/tokenizer_config.json

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/zhou/.conda/envs/medicalgpt/lib/python3.9/site-packages/transformers/utils/hub.py:417 in   │
│ cached_file                                                                                      │
│                                                                                                  │
│    414 │   user_agent = http_user_agent(user_agent)                                              │
│    415 │   try:                                                                                  │
│    416 │   │   # Load from URL or cache if already cached                                        │
│ ❱  417 │   │   resolved_file = hf_hub_download(                                                  │
│    418 │   │   │   path_or_repo_id,                                                              │
│    419 │   │   │   filename,                                                                     │
│    420 │   │   │   subfolder=None if len(subfolder) == 0 else subfolder,                         │
│                                                                                                  │
│ /home/zhou/.conda/envs/medicalgpt/lib/python3.9/site-packages/huggingface_hub/utils/_validators. │
│ py:120 in _inner_fn                                                                              │
│                                                            

In [3]:
import os
os.environ["RWKV_JIT_ON"] = '1'
os.environ["RWKV_CUDA_ON"] = '0' # if '1' then use CUDA kernel for seq mode (much faster)
from rwkv.model import RWKV                         # pip install rwkv
model = RWKV(model='/fsx/BlinkDL/HF-MODEL/rwkv-4-pile-1b5/RWKV-4-Pile-1B5-20220903-8040', strategy='cuda fp16')

out, state = model.forward([187, 510, 1563, 310, 247], None)   # use 20B_tokenizer.json
print(out.detach().cpu().numpy())                   # get logits
out, state = model.forward([187, 510], None)
out, state = model.forward([1563], state)           # RNN has state (use deepcopy if you want to clone it)
out, state = model.forward([310, 247], state)
print(out.detach().cpu().numpy())                   # same result as above

TypeError: __init__() missing 2 required positional arguments: 'model' and 'strategy'

In [6]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('moka-ai/m3e-base')

#Our sentences we like to encode
sentences = [
    '* Moka 此文本嵌入模型由 MokaAI 训练并开源，训练脚本使用 uniem',
    '* Massive 此文本嵌入模型通过**千万级**的中文句对数据集进行训练',
    '* Mixed 此文本嵌入模型支持中英双语的同质文本相似度计算，异质文本检索等功能，未来还会支持代码检索，ALL in one'
]

#Sentences are encoded by calling model.encode()
embeddings = model.encode(sentences)

#Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))